In [20]:
import pandas as pd
import numpy as np


import fitz
from camel_tools.utils.dediac import dediac_ar
from ar_corrector.corrector import Corrector
from deep_translator import GoogleTranslator

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from summarizer import Summarizer,TransformerSummarizer


In [6]:
def convert_pdf_to_text(path):
    with fitz.open(path) as doc:
        for page in doc:
            blocks = page.get_text_blocks()
            corr = Corrector()
            texts = []
            for block in blocks:
                text = block[4:-2][0]
                text = dediac_ar(text[::-1]).replace('�','').strip('\n').replace('\n','')
                corrected = corr.contextual_correct(text)
                texts.append(corrected)
    return texts


In [7]:
def translate_texts(text):
    translator = GoogleTranslator(source='ar', target='en')
    translated = translator.translate(text)

    return translated

In [8]:
def filtering_events(texts):
    
    ps = PorterStemmer()
    key_words = ['occupation','sit-in','demonstration','protest','roadblock','blockade','attack','march','strike']
    key_words = [ps.stem(word) for word in key_words]
    
    # texts = [word_tokenize(text) for text in texts]
    
    events = []
    not_events = []
    for text in texts:
        if any(x in [ps.stem(word) for word in word_tokenize(text)] for x in key_words):
            events.append(text)
        else:
            not_events.append(text) 
        
    return events, not_events

In [9]:
path = '/home/noahdarwich/code/NoahDarwich/pdf_coder/raw_data/20194004.1.pdf'
texts = convert_pdf_to_text(path)
all_texts = [translate_texts(text) for text in texts]

In [10]:
events, not_events = filtering_events(all_texts)

In [11]:
df = pd.read_csv('../raw_data/tishreen_full.csv')
train_df = df[df['URL1'] == 'https://tareeqashaab.com/images/TariqPDF/2019/10/3.pdf']
target_df = [translate_texts(text) for text in list(train_df['narrative'])]

In [12]:
target_df = [(x,1) for x in target_df]
target_df

[('A demonstration in Tahrir Square calling for the eradication of corruption and unemployment, the security forces dispersed the demonstrators using live bullets and tear gas',
  1),
 ('Angry protests in the Zafaraniya area of \u200b\u200bBaghdad due to the security forces cutting off the roads leading to Tahrir Square',
  1),
 ('Angry protests in Al-Sadr area in Baghdad due to the security forces cutting off the roads leading to Tahrir Square',
  1),
 ('Angry protests in the Husseiniya area of \u200b\u200bBaghdad due to the security forces cutting off the roads leading to Tahrir Square',
  1),
 ('Angry protests in Al-Shaab area in Baghdad due to the security forces cutting off the roads leading to Tahrir Square',
  1),
 ('Angry protests in the Bob Al-Sham area of \u200b\u200bBaghdad due to the security forces cutting off the roads leading to Tahrir Square',
  1),
 ('Angry protests in the Al-Obaidi area of \u200b\u200bBaghdad due to the security forces cutting off the roads leading to

In [26]:
# model = Summarizer()
# summary_ = [model(text, min_length=3) for text in not_events]
# summary_ 

In [27]:
# model = TransformerSummarizer(transformer_type="XLNet",transformer_model_key="xlnet-base-cased")
# summary_2 = [model(text, min_length=3) for text in not_events]
# summary_2